##### From: https://developers.arcgis.com/python/samples/inventory-organizational-content/

##### Imports

In [1]:
import pandas as pd
from IPython.display import display
import arcgis
from arcgis.gis import GIS
import re

##### Connect

In [3]:
# Portal organization URL

# BUILT IN PSA
url = 'https://<MY.SERVER.COM>:7443/arcgis'

# Built in PSA
username = '<PSA>'
password = '<PSA PASSWORD>'

# Max users returned by Portal user search (set higher than Portal user count)
max_users = 5000

# BUILT IN PSA
gis = GIS(url, username, password)

##### Get users

In [4]:
org_users = gis.users.search(max_users=max_users, exclude_system=True)
print(f'{len(org_users)} users found')

2 users found


##### Get content

In [5]:
org_content = []

for user in org_users:
    qe = f"owner: {user.username}"
    user_content = gis.content.advanced_search(query=qe, max_items=-1)['results']
    org_content += user_content
    
content_df = pd.DataFrame(org_content)

# ITEM COUNT BY TYPE
print(content_df.type.value_counts().head(10))

print('-------------------------')

# ITEM COUNT BY ACCESS LEVEL
print(content_df.access.value_counts().head(10))

type
Vector Tile Service      6
Vector Tile Package      6
Geoprocessing Service    4
Map Service              3
Geometry Service         1
Feature Service          1
Feature Collection       1
WMS                      1
Name: count, dtype: int64
-------------------------
access
private    19
org         4
Name: count, dtype: int64


##### Filter columns

In [6]:
view_columns = ['id', 'title', 'owner', 'ownerFolder', 'type', 'access', 'created','modified','numViews','numRatings','avgRating','numComments','groupDesignations','url','typeKeywords']

# ITEM LIST - FILTER COLUMNS
print(content_df[view_columns].head(10))

                                 id  ...                                       typeKeywords
0  5e8471b650e04bb699ed75a40ae1659a  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
1  2f763b9eaef245adae4e7c4a802122c8  ...      [ArcGIS Pro, Data, Vector Tile Package, vtpk]
2  3407760fd4624b1cb4f2a43ed646dc39  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
3  7cd28495b3444b50b7ed118174e57bac  ...      [ArcGIS Pro, Data, Vector Tile Package, vtpk]
4  70b72c72fa424ad8a935dc8165c00afc  ...  [ArcGIS Server, Geoprocessing Service, Service...
5  363e09dd504d4bd0bf9d27a29fdfbf0a  ...   [ArcGIS Server, Geometry Service, Service, Tool]
6  f481f08bb3eb491eb09e33402b514871  ...  [ArcGIS Server, Data, Feature Access, Feature ...
7  3d0d205fde034fcb80dd82dba4b5f980  ...  [Data, Feature Collection, Map Notes, Multilayer]
8  fbcb79fc1fad4c1eb1fccde9f74ad955  ...  [ArcGIS Server, Data, Map Service, Metadata, S...
9  894680d58333430d904623be99209538  ...  [ArcGIS Server, Data, Map Service, Met

##### Get vector tile serivce items. Return subset of columns.

*THESE ARE CANDIDATES FOR MANUAL REPAIR.*

In [7]:
#web_map_filter = content_df.numViews < 5

#https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types.htm
#type_filter = content_df.type=='Feature Service'
#type_filter = content_df.typeKeywords.str.contains('Hosted Service',regex=False)
type_filter = content_df.typeKeywords.str.contains('Vector Tile Service',regex=False)

#combined_mask = web_map_filter & type_filter
combined_mask = type_filter

print(content_df[combined_mask][view_columns].head(20))

                                  id  ...                                       typeKeywords
0   5e8471b650e04bb699ed75a40ae1659a  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
2   3407760fd4624b1cb4f2a43ed646dc39  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
15  618efa9564c94fa2946acee6d3e6a4fc  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
17  724d63a5ea894b7881fbf941c682a262  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
19  a81a125c2a594260905ba8613885ce20  ...  [Data, Service, Vector Tile Service, ArcGIS Se...
21  3977369364574534b19ff8ebadd63a6e  ...  [Data, Service, Vector Tile Service, ArcGIS Se...

[6 rows x 15 columns]


##### Export to CSV

*CONSIDERATIONS FOR REVIEWING THE RESULTING CSV FILE IN EXCEL*
- Convert date in Excel using =DATE(1970,1,1+I2/86400000)
- Consider ordering by the following fields when reviewing content to be marked for deletion:owner, access, numViews, modified, created
- Note that hosted services are identified by the typeKeyword 'Hosted Service'

In [8]:
target_path = "c:\\temp\\org_content2.csv" 
content_df[combined_mask][view_columns].to_csv(target_path)